# Prep data

In [2]:
%sh
set -x
mkdir -p $HOME/pet_data
cd $HOME/pet_data
curl -fsO http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
curl -fsO http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
tar -xf images.tar.gz
tar -xf annotations.tar.gz

+ mkdir -p /root/pet_data
+ cd /root/pet_data
+ curl -fsO http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
+ curl -fsO http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
+ tar -xf images.tar.gz
+ tar -xf annotations.tar.gz

In [3]:
%sh
# Note: It is normal to see some warnings when running this script. You may ignore them.
cd /opt/tensorflow/models/research
mkdir -p /dbfs/object_detection/inception/data
cp object_detection/data/pet_label_map.pbtxt /dbfs/object_detection/inception/data
PYTHONPATH=. /databricks/python/bin/python object_detection/dataset_tools/create_pet_tf_record.py \
    --label_map_path=/dbfs/object_detection/inception/data/pet_label_map.pbtxt \
    --data_dir=$HOME/pet_data \
    --output_dir=/dbfs/object_detection/inception/data

/opt/tensorflow/models/research/object_detection/utils/dataset_util.py:75: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
 if not xml:
WARNING:root:Could not find /root/pet_data/annotations/xmls/Egyptian_Mau_14.xml, ignoring example.
WARNING:root:Could not find /root/pet_data/annotations/xmls/saint_bernard_15.xml, ignoring example.
WARNING:root:Could not find /root/pet_data/annotations/xmls/Ragdoll_199.xml, ignoring example.
WARNING:root:Could not find /root/pet_data/annotations/xmls/Egyptian_Mau_186.xml, ignoring example.
WARNING:root:Could not find /root/pet_data/annotations/xmls/Bengal_175.xml, ignoring example.
WARNING:root:Could not find /root/pet_data/annotations/xmls/samoyed_10.xml, ignoring example.
WARNING:root:Could not find /root/pet_data/annotations/xmls/Egyptian_Mau_156.xml, ignoring example.
WARNING:root:Could not find /root/pet_data/annotations/xmls/Abyssinian_104.xml, ignoring example.
WARNING:root:Could not find /root/pet_data/annotations/xmls/Bengal_111.xml, ignoring example.

# Get model and create job config

In [5]:
%sh
set -x

model=faster_rcnn_inception_v2_coco_2018_01_28
test -e $model.tar.gz || curl -fsO http://download.tensorflow.org/models/object_detection/$model.tar.gz
tar -xvf $model.tar.gz --no-same-owner 
cp $model/model.ckpt.* /dbfs/object_detection/inception/data

# Edit the job config template. Please note that there
# are multiple places where PATH_TO_BE_CONFIGURED needs to be set.
sed -e "s|PATH_TO_BE_CONFIGURED|/dbfs/object_detection/inception/data|g" \
    /opt/tensorflow/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config \
    > /dbfs/object_detection/inception/object_detection.config

+ model=faster_rcnn_inception_v2_coco_2018_01_28
+ test -e faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
+ curl -fsO http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
+ tar -xvf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz --no-same-owner
faster_rcnn_inception_v2_coco_2018_01_28/
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.index
faster_rcnn_inception_v2_coco_2018_01_28/checkpoint
faster_rcnn_inception_v2_coco_2018_01_28/pipeline.config
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.meta
faster_rcnn_inception_v2_coco_2018_01_28/saved_model/
faster_rcnn_inception_v2_coco_2018_01_28/saved_model/saved_model.pb
faster_rcnn_inception_v2_coco_2018_01_28/saved_model/variables/
faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb
+ cp faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001 faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.index faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.meta /dbfs/object_detection/inception/data
+ sed -e 's|PATH_TO_BE_CONFIGURED|/dbfs/object_detection/inception/data|g' -e 's|initial_crop_size:|second_stage_batch_size: 4\n&|' -e 's|max_dimension: 1024|max_dimension: 600|' -e 's|max_total_detections: 300|max_total_detections: 100|' /opt/tensorflow/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config

# Config and run job
Run the following cell and open the View TensorBoard link to monitor progress.

In [7]:
dbutils.tensorboard.start("/root/object_detection/inception")

TensorBoard log directory set to: object_detection/inception. 
 View TensorBoard

In [8]:
%sh
cd /opt/tensorflow/models/research/
PYTHONPATH=.:./slim /databricks/python/bin/python object_detection/model_main.py --logtostderr --model_dir=$HOME/object_detection/inception --pipeline_config_path=/dbfs/object_detection/inception/object_detection.config

WARNING:tensorflow:Forced number of epochs for all eval validations to be 1.
W0529 12:28:29.750766 140715813521152 tf_logging.py:125] Forced number of epochs for all eval validations to be 1.
WARNING:tensorflow:Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
W0529 12:28:29.751054 140715813521152 tf_logging.py:125] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
WARNING:tensorflow:Estimator's model_fn (<function create_model_fn.<locals>.model_fn at 0x7ffa702cf620>) includes params argument, but params are not passed to Estimator.
W0529 12:28:29.751601 140715813521152 tf_logging.py:125] Estimator's model_fn (<function create_model_fn.<locals>.model_fn at 0x7ffa702cf620>) includes params argument, but params are not passed to Estimator.
WARNING:tensorflow:num_readers has been reduced to 10 to match input file shards.
W0529 12:28:29.981210 140715813521152 tf_logging.py:125] num_readers has been reduced to 10 to match input file shards.
WARNING:tensorflow:From /opt/tensorflow/models/research/object_detection/builders/dataset_builder.py:80: parallel_interleave (from tensorflow.contrib.data.python.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
W0529 12:28:30.056198 140715813521152 tf_logging.py:125] From /opt/tensorflow/models/research/object_detection/builders/dataset_builder.py:80: parallel_interleave (from tensorflow.contrib.data.python.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
WARNING:tensorflow:From /databricks/python/lib/python3.6/site-packages/tensorflow/python/ops/sparse_ops.py:1165: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W0529 12:28:30.198476 140715813521152 tf_logging.py:125] From /databricks/python/lib/python3.6/site-packages/tensorflow/python/ops/sparse_ops.py:1165: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
WARNING:tensorflow:From /opt/tensorflow/models/research/object_detection/builders/dataset_builder.py:148: batch_and_drop_remainder (from tensorflow.contrib.data.python.ops.batching) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.batch(..., drop_remainder=True)`.
W0529 12:28:30.807924 140715813521152 tf_logging.py:125] From /opt/tensorflow/models/research/object_detection/builders/dataset_builder.py:148: batch_and_drop_remainder (from tensorflow.contrib.data.python.ops.batching) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.batch(..., drop_remainder=True)`.
WARNING:tensorflow:From /opt/tensorflow/models/research/object_detection/predictors/heads/box_head.py:93: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
W0529 12:28:34.298299 140715813521152 tf_logging.py:125] From /opt/tensorflow/models/research/object_detection/predictors/heads/box_head.py:93: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
WARNING:tensorflow:From /opt/tensorflow/models/research/object_detection/meta_architectures/faster_rcnn_meta_arch.py:2164: get_or_create_global_step (from tensorflow.contrib.framew